## CSE 255 Problem Set 1: Collinear points

For this problem set, we'll be using the Jupyter notebook:

![](jupyter.png)

### In this problem set, you will write python3 code to find sets of collinear points given __ 2D points.

Definition of collinearity: In geometry, collinearity of a set of points is the property of their lying on a single line. A set of points with this property is said to be collinear.

![](non-collinear-points.jpg)

Here, points P,Q,R and A,R,B are collinear. However, points A,B,C are non-collinear.

Given an input file with a set of co-ordinates, your task is to use pyspark library functions to write a program to find if three points are collinear.

For instance, if given these points: {(1,1), (0,1), (2,2), (3,3), (0,5), (3,4), (5,6), (0,-3), (-2,-2)}

Sets of collinear points are: {((-2,-2), (1,1), (2,2), (3,3)), ((0,1), (3,4), (5,6)), ((0,-3), (0,1), (0,5))}. Note that the points within each set needs to be in sorted order. However, the ordering of the sets does not matter. 

Note: Every set has to have atleast three points (single point or a pair of points always lie on a line).

1. https://en.wikipedia.org/wiki/Collinearity
2. http://www.mathcaptain.com/geometry/collinear-points.html

## Points of concerns for TAs (keep adding stuff as you go along)

1. What should be the input format? Should we give them a file containing a bunch of 2D points?
2. Do we let them import any library they want or enforce it?


In [14]:
## generate data points that go in the data input file for students

from itertools import product
import numpy as np

l = list(product(range(-400,400,np.random.randint(21)), range(-400,400,np.random.randint(20))))

In [2]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("Collinear Points").setMaster("local[4]")
sc = SparkContext(conf=conf)

---
## Part A: Load the data (2 points)


Write a function that returns a list of numbers, such that $x_i=i^2$, for $1\leq i \leq n$. Make sure it handles the case where $n<1$ by raising a `ValueError`.

In [8]:
### BEGIN SOLUTION
def func(x):
    arr = x.split()
    return tuple([int(el) for el in arr])

def filt(x):
    print(x)
    return x[0] != x[1]

def findslope(x):
    if x[1][0] - x[0][0] == 0:
        slope = "inf"
    else:
        slope = 1.0 * (x[1][1] - x[0][1])/(x[1][0] - x[0][0])
    return ((x[0], slope), (x[1]))

def returnpoints(x):
#     print(x[0])
    x[1].append(x[0][0])
    arr = sorted(x[1])
    return tuple(arr)
#     return list(x[1]).append(x[0][0])

def removedups(x, y):
    return set((x,y))
### END SOLUTION

def get_collinear(filename):
    """Compute the set of collinear points where each set of points is sorted"""
    rdd = sc.textFile(filename)
    ### BEGIN SOLUTION
    rdd1 = rdd.map(func)
    rdd1 = rdd1.cartesian(rdd1)
    rdd1 = rdd1.filter(filt).map(findslope).groupByKey().mapValues(list).filter(lambda x: len(x[1]) > 1)
    rdd1 = rdd1.map(returnpoints)
    res = set(rdd1.collect())
    return res
    ### END SOLUTION

Your function should print `[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]` for $n=10$. Check that it does:

In [19]:
"""Check that the get_collinear function returns the correct output for several all input files"""
assert get_collinear("data.txt") == {((-2, -2), (1, 1), (2, 2), (3, 3)), ((0, 1), (3, 4), (5, 6)), ((0, -3), (0, 1), (0, 5))}

### BEGIN HIDDEN TESTS

### END HIDDEN TESTS

In [13]:
"""Check that the get_collinear function raises an error for invalid input files"""
try:
    get_collinear("data.txt")
except ValueError:
    pass
else:
    raise AssertionError("did not raise")

AssertionError: did not raise